In [1]:
from bs4 import BeautifulSoup
import sys
from urllib.parse import urlparse

In [10]:
if 'ipykernel' in sys.modules:
    sys.argv = [
        sys.argv[0],
        "/mnt/c/Users/tynda/Downloads/Russell, James C - The Germanization of early medieval Christianity _ a sociohistorical approach to religious transformation-Oxford University Press (1994).epub",
    ]

pathOrUrl = sys.argv[1]

if pathOrUrl.startswith('http'):
    parsed_url = urlparse(pathOrUrl)
    path = parsed_url.path

    # Split the path by slashes and take the last element
    if path[-1] == '/':
        path = path[:-1]

    output_file_stem = path.split('/')[-1]

    # Fetch the HTML content
    import requests
    response = requests.get(pathOrUrl)
    html_content = response.text
else:
    from pathlib import Path
    path = Path(pathOrUrl)
    output_file_stem = path.stem
    with open(pathOrUrl) as f:
        print(sys.version)
        print(f'{path.suffix=}')
        if path.suffix == '.epub':
            from ebooklib import epub
            import ebooklib
            book = epub.read_epub(pathOrUrl)
            text = []
            for item in book.get_items():
                if item.get_type() == ebooklib.ITEM_DOCUMENT:
                    soup = BeautifulSoup(item.get_content(), 'html.parser')
                    text.append(soup.get_text())
        
            html_content = '\n'.join(text)
        else:
            html_content = f.read()


3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]
path.suffix='.epub'


In [11]:
# Parse the HTML content with Beautiful Soup
soup = BeautifulSoup(html_content, 'html.parser')

print(f"{len(sys.argv)=}")
if len(sys.argv) == 2:
    article = soup
else:
    article = soup.find(sys.argv[2]) if len(sys.argv) < 4 else soup.find(sys.argv[2], class_=sys.argv[3])
# article = soup.find(sys.argv[2], class_=sys.argv[3])

if not article:
    print("no atricle element")
    body = soup.body
    # for header in body.find_all('script'):
    #     header.decompose()
    
    # for script in body.find_all('script'):
    #     script.decompose()

    # for svg in body.find_all('svg'):
    #     svg.decompose()

    print(body)
    print(sys.argv[1:])
    exit(1)

# Find and remove all <code> elements
for code_tag in article.find_all(['pre', 'math', 'code']):
    code_tag.decompose()

# Extract text and remove leading/trailing whitespace
text_content = article.get_text(separator='\n', strip=True)
print(text_content)

len(sys.argv)=2
THE GERMANIZATION OF EARLY MEDIEVAL CHRISTIANITY








The Germanization of Early Medieval Christianity
A Sociohistorical Approach to Religious Transformation
James C. Russell








Oxford University Press
Oxford New YorkAthens Auckland Bangkok BombayCalcutta Cape Town Dar es Salaam DelhiFlorence Hong Kong Istanbul KarachiKuala Lumpur Madras Madrid MelbourneMexico City Nairobi Paris SingaporeTaipei Tokyo Toronto
and associated companiesin Berlin Ibadan
Copyright © 1994 by James C. Russell
First published in 1994 by Oxford University Press, Inc.,198 Madison Avenue, New York, New York 10016
First issued as an Oxford University Press paperback, 1996.
Oxford is a registered trademark of Oxford University Press, Inc.
All rights reserved. No part of this publication may be reproduced,stored in a retrieval system, or transmitted, in any form or by any means,electronic, mechanical, photocopying, recording, or otherwise,without the prior permission of Oxford University Press

In [12]:
from nltk.data import load
from TTS.api import TTS
tts = TTS(model_name="tts_models/en/vctk/vits", gpu=True)
sample_rate = tts.synthesizer.output_sample_rate

/home/a/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/a/.local/lib/python3.12/site-packages/TTS/api.py:91: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


In [13]:
tokenizer = load(f"tokenizers/punkt/english.pickle")
sentences = tokenizer.tokenize(text_content)
print(sentences)

['THE GERMANIZATION OF EARLY MEDIEVAL CHRISTIANITY\n\n\n\n\n\n\n\n\nThe Germanization of Early Medieval Christianity\nA Sociohistorical Approach to Religious Transformation\nJames C. Russell\n\n\n\n\n\n\n\n\nOxford University Press\nOxford New YorkAthens Auckland Bangkok BombayCalcutta Cape Town Dar es Salaam DelhiFlorence Hong Kong Istanbul KarachiKuala Lumpur Madras Madrid MelbourneMexico City Nairobi Paris SingaporeTaipei Tokyo Toronto\nand associated companiesin Berlin Ibadan\nCopyright © 1994 by James C. Russell\nFirst published in 1994 by Oxford University Press, Inc.,198 Madison Avenue, New York, New York 10016\nFirst issued as an Oxford University Press paperback, 1996.', 'Oxford is a registered trademark of Oxford University Press, Inc.\nAll rights reserved.', 'No part of this publication may be reproduced,stored in a retrieval system, or transmitted, in any form or by any means,electronic, mechanical, photocopying, recording, or otherwise,without the prior permission of Oxfor

In [13]:
import subprocess
from TTS.tts.utils.synthesis import synthesis

command = [
            'ffmpeg', '-y',
            '-f', 'f32le', 
            '-ar', str(sample_rate),  # sample rate
            '-ac', '1',  # number of audio channels
            '-i', '-',  # The input comes from stdin
            '-acodec', 'copy',  # audio codec for M4A
            output_file_stem + '.wav',
        ]
print(" ".join(command))

process = subprocess.Popen(command, stdin=subprocess.PIPE)

for sentence in sentences:
    wav = synthesis(
            model=tts.synthesizer.tts_model,
            text=sentence,
            CONFIG=tts.synthesizer.tts_config,
            use_cuda=True,
            speaker_id=77, # p307
            use_griffin_lim=tts.synthesizer.vocoder_model is None,
            )["wav"]
    format = wav
    process.stdin.write(wav.tobytes())

process.stdin.close()
process.wait()
print("------------\n", format)

ffmpeg -y -f f32le -ar 22050 -ac 1 -i - -acodec copy blog.wav


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

------------
 [-0.00024886 -0.00042764 -0.00031147 ...  0.00055181  0.00037304
  0.00013643]


size=   27012kB time=00:05:13.60 bitrate= 705.6kbits/s speed=6.31x    
video:0kB audio:27012kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000412%


In [19]:
print(sys.argv)

['/home/a/.local/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/a/.local/share/jupyter/runtime/kernel-v2-17921kGKUpKVYOTJo.json']
